In [ ]:
### Import Libraries.

library("scMetabolism")
library("ggplot2")
library("dplyr")
library("rsvd")
library("Seurat")
library("RColorBrewer")
library("qs")
library("reshape2")
library("data.table")
library("grid")

In [ ]:
### Load Data.

setwd("/folder")
Seurat <- qread("Seurat.qs")
Seurat

In [ ]:
### Duplicate and Reassign RNA assay. Set RNA3 as default.

Seurat[["RNA3"]] <- as(object = Seurat[["RNA"]], Class = "Assay")
Seurat[["RNA5"]] <- Seurat[["RNA"]]
Seurat[["RNA"]] <- Seurat[["RNA3"]]
DefaultAssay(Seurat) <- "RNA"

In [ ]:
### scMetabolism Analysis – KEGG and REACTOME

# KEGG
Seurat_Kegg <- sc.metabolism.Seurat(
  obj = Seurat,
  method = "AUCell",
  imputation = FALSE,
  metabolism.type = "KEGG"
)

# REACTOME
Seurat_Reactome <- sc.metabolism.Seurat(
  obj = Seurat,
  method = "AUCell",
  imputation = FALSE,
  metabolism.type = "REACTOME"
)

In [ ]:
### Extract Metabolism Matrix.

metabolism.matrix <- Seurat_Kegg@assays$METABOLISM$score ### Or Seurat_Reactome

DimPlot.metabolism(
  obj = Seurat_Kegg,
  pathway = "Metabolism of nucleotides",
  dimention.reduction.type = "umap",
  dimention.reduction.run = FALSE,
  size = 1
)

In [ ]:
### DotPlot.

input.pathway <- c("Pathway_1", "Pathway_2", "Pathway_3", "Pathway_4")

DotPlot.metabolism(
  obj = Seurat_Kegg, ### Or Seurat_Reactome
  pathway = input.pathway,
  phenotype = "Cluster_Column",
  norm = "y"
)

In [ ]:
### BoxPlot.

input.pathway <- c("Pathway_1", "Pathway_2", "Pathway_3", "Pathway_4")

BoxPlot.metabolism(
  obj = Seurat_Kegg, ### Or Seurat_Reactome
  pathway = input.pathway,
  phenotype = "Cluster_Column",
  ncol = 1
)

In [ ]:
### Function: DotPlot.

DotPlot.metabolism.df <- function(df, pathway, phenotype, norm = "y") {
  
  df <- as.data.frame(df)

  desired_levels_phenotype <- c(
    "Cluster_1", "Cluster_2", "Cluster_3", "Cluster_4"
  )

  desired_levels_pathway <- c(
    "Pathway_1", "Pathway_2", "Pathway_3", "Pathway_4"
  )

  df_sub <- df[df$Pathways %in% pathway, ]
  if (nrow(df_sub) == 0) {
    stop("No matching pathways found. Check spelling.")
  }

  df_sub <- df_sub[, c(phenotype, "Pathways", "Score"), drop = FALSE]
  colnames(df_sub) <- c("Phenotype", "Pathway", "Score")

  df_sub$Phenotype <- factor(df_sub$Phenotype, levels = desired_levels_phenotype)
  df_sub$Pathway <- factor(df_sub$Pathway, levels = desired_levels_pathway)

  gg_table_median <- aggregate(
    Score ~ Phenotype + Pathway,
    data = df_sub,
    FUN = median
  )

  range01 <- function(x) (x - min(x)) / (max(x) - min(x))

  if (norm == "y") {
    gg_table_median <- do.call(rbind, lapply(
      split(gg_table_median, gg_table_median$Pathway),
      function(subdf) {
        subdf$Score <- range01(subdf$Score)
        subdf
      }
    ))
  } else if (norm == "x") {
    gg_table_median <- do.call(rbind, lapply(
      split(gg_table_median, gg_table_median$Phenotype),
      function(subdf) {
        subdf$Score <- range01(subdf$Score)
        subdf
      }
    ))
  }

  ggplot(gg_table_median, aes(
    x = Phenotype, y = Pathway,
    color = Score, size = Score
  )) +
    geom_point() +
    ylab("Metabolic Pathway") +
    xlab(phenotype) +
    labs(title = "Dot Plot of Metabolic Pathways") +
    scale_colour_gradientn(colours = rev(brewer.pal(n = 11, name = "Spectral"))) +
    scale_size(range = c(0, 10), limits = c(0, 1)) +
    theme(
      panel.background = element_rect(fill = "white"),
      plot.background = element_rect(fill = "white"),
      axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
      axis.title.x = element_text(size = 15),
      axis.title.y = element_text(size = 15),
      legend.text = element_text(size = 13),
      legend.title = element_text(size = 10),
      legend.key.size = unit(0.7, "cm"),
      plot.title = element_text(size = 15),
      strip.text = element_text(size = 10, face = "bold", color = "white"),
      strip.background = element_rect(fill = "#555555", color = NA),
      legend.position = "none"
    )
}

plot <- DotPlot.metabolism.df(
  merged_df,
  pathway = unique(merged_df$Pathways),
  phenotype = "Cluster_Column",
  norm = "y"
)
plot